In [ ]:
# Exports QuantumCollocation, NamedTrajectories, and TrajectoryIndexingUtils
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils
using LinearAlgebra

# Plots
using CairoMakie
using LaTeXStrings

In [ ]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units

const n_qubits = 2
const n_levels = 2

t_f = 10 * ns
n_steps = 51
times = range(0, t_f, n_steps)  # Alternative: collect(0:Δt:t_f)
Δt = times[2] - times[1]

In [ ]:
# Operators
Paulis = Dict(
    "I" => Matrix{ComplexF64}(I, 2, 2),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 -im; im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)
X0=kron(Paulis["X"],Paulis["I"])
Y0=kron(Paulis["Y"],Paulis["I"])
X1=kron(Paulis["I"],Paulis["X"])
Y1=kron(Paulis["I"],Paulis["Y"])

excitation(theta) = exp(-im/2 * theta * (X0 * Y1 - Y0 * X1))

In [ ]:
a = [0im 1; 0 0]
ad = transpose(a)
a0  = kron(a,Paulis["I"])
ad0  = kron(ad,Paulis["I"])
a1  = kron(Paulis["I"],a)
ad1  = kron(Paulis["I"],ad);

# instantiate an identity matrix
I2 = Matrix{ComplexF64}(I, 2, 2);

In [ ]:
H_drift = zeros(ComplexF64, n_levels^n_qubits, n_levels^n_qubits)

H_drives = [
    Matrix{ComplexF64}(I, n_levels^n_qubits, n_levels^n_qubits),
    ad0 * a0,
    ad1 * a1,
    a0 + ad0,
    im * (a0 - ad0),
    a1 + ad1,
    im * (a1 - ad1),
    ad0 * a1 + a0 * ad1,
    # -im * (ad0 * a1 - a0 * ad1),
    # (a0 + ad0) * (a1 + ad1),
    # -(a0 - ad0) * (a1 - ad1)
]
system = QuantumSystem(H_drift, H_drives)
;

The Hamiltonian we use to drive the gate is the same as the excitation.

In [ ]:
# theta = 1.0 * pi
theta = 1.15 * pi

target = excitation(theta)

target == exp(-im * theta * -im * (ad0 * a1 - a0 * ad1))

It looks like the path of the optimizer goes through these regions where inf_du is saturated. The objective continues to decrease, even if the inf_du is sitting at a value that is not optimal.

We are observing that the objective is not senstive to the constraints.

We can add a heuristic from ipopt to help.

In [ ]:
# Shape the cost function with weights on states and controls
Q = 10.
R = 1e-4
# Add control bounds
a_bound = 2 * π * 500 * MHz
dda_bound = 0.5

ops = Options()
ops.print_info_string = "yes"
ops.recalc_y = "yes"
ops.recalc_y_feas_tol = 1.0

p = UnitarySmoothPulseProblem(
    system,
    target,
    n_steps,
    Δt;
    Δt_min = 0.5 * Δt,
    Δt_max = 1.5 * Δt,
    a_bound=a_bound,
    dda_bound=dda_bound,
    Q=Q,
    R=R,
    verbose=true,
    hessian_approximation=true,
    # pade_order=10,
    pade_order=20,
    free_time=false,
    timesteps_all_equal=true,
    ipopt_options=ops,
)

In [ ]:
solve!(p; max_iter=50)
result = copy(p.trajectory)

println("Fidelity: ", unitary_fidelity(p))

In [ ]:
plot(result)

# Tests

In [ ]:
# Test
states = map(iso_vec_to_operator, eachslice(result[:Ũ⃗], dims=2))
println("Fidelity ", unitary_fidelity(p))
println("Stored fidelity ", unitary_fidelity(result[:Ũ⃗][:, end], operator_to_iso_vec(target)))


rollout_states = unitary_rollout(result, system; integrator=exp)
ΔUs = rollout_states .- result[:Ũ⃗]
norm_ΔUs = map(norm, eachslice(rollout_states .- result[:Ũ⃗], dims=2))
fid_ΔUs = [unitary_fidelity(rollout_states[:, i], result[:Ũ⃗][:, i]) for i ∈ 1:size(result[:Ũ⃗], 2)]

println("Rollout Error ", maximum(norm_ΔUs))

unitarity = []
for Ũ⃗ in eachcol(rollout_states)
    U = iso_vec_to_operator(Ũ⃗)
    d = diag(U'U)
    push!(unitarity, (maximum(real.(d)), maximum(imag.(d))))
end
unitarity

In [ ]:
f = Figure(resolution = 1.5 .* (525, 325))
ax1 = f[1,1] = Axis(f, xlabel = "step", ylabel = L"$||\Delta U||$")
ax2 = f[1,1] = Axis(f, xlabel = "step", ylabel = L"$\ddot{a}$")
ax2.yaxisposition = :right
ax2.yticklabelalign = (:left, :center)
ax2.xticklabelsvisible = false
ax2.xlabelvisible = false

lines!(ax1, 1 .- fid_ΔUs, linewidth=3, color=:black, linestyle=:solid)
# lines!(ax1, accumulate(+, 1 .- fid_ΔUs), linewidth=3, color=:black, linestyle=:solid)


# lines!(ax2, -norm_ΔUs, linewidth=3, color=:black, linestyle=:solid)
# lines!(ax1, accumulate(+, -norm_ΔUs), linewidth=3, color=:black, linestyle=:solid)

# for row in eachrow(ΔUs)
#     # lines!(ax1, accumulate(+, row), linewidth=2, linestyle=:solid)
#     lines!(ax1, row, linewidth=2, linestyle=:solid)
# end

for row in eachrow(result[:a])
    lines!(ax2, row, linewidth=2, linestyle=:dash)
end
# lines!(ax2, result[:a][end, :], linestyle=:dash, linewidth=3, color=:black)

# linkxaxes!(ax1,ax2)
f

U_tests = unitary_rollout(a, Δt, p.system)

ΔUs_test = U_tests .- result[:Ũ⃗]
norm_ΔUs_test = map(norm, eachslice(U_tests .- result[:Ũ⃗], dims=2))
fid_ΔUs_test = [unitary_fidelity(U_tests[:, i], result[:Ũ⃗][:, i]) for i ∈ 1:size(result[:Ũ⃗], 2)];

f = Figure(resolution = 1.5 .* (525, 325))
ax1 = f[1,1] = Axis(f, xlabel = "step", ylabel = L"$||\Delta U||$")
ax2 = f[1,1] = Axis(f, xlabel = "step", ylabel = L"$\ddot{a}$")
ax2.yaxisposition = :right
ax2.yticklabelalign = (:left, :center)
ax2.xticklabelsvisible = false
ax2.xlabelvisible = false

lines!(ax1, 1 .- fid_ΔUs_test, linewidth=3, color=:black, linestyle=:solid)

lines!(ax1, 1 .- fid_ΔUs, linewidth=3, color=:orange, linestyle=:solid)


f

f = Figure(resolution = (525, 325))
ax = Axis(f[1, 1])
hist!(ax, timesteps(result))
f